This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/pend.html).  I recommend having both windows open, side-by-side!


In [2]:
import numpy as np
from pydrake.all import StartMeshcat

from underactuated.meshcat_utils import interact

In [3]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


# Recurrent neural network units: LSTM and JANET

JANET (Just Another NETwork) is a simplied version of the famous Long Short-Term Memory (LSTM) model as described in https://arxiv.org/abs/1804.04849 .  Relative to the autapse model, it adds a multiplicative "forget gate":
$$\dot{x} + x = f (1-\alpha) x + (1-f)\tanh(wx + u),\\ f = \sigma(w_f x + u_f).$$
I've written it here in continuous time and also added a small "leak term", $\alpha>0$.  $\sigma()$ is the sigmoid function, with range (0,1).  When $f=0$ we have the autapse model, where "memory" is possible.  When $f=1$ we have $\dot{x} = -\alpha x$, which "forgets" by implementing a dynamics with pushing the activations back towards zero.  

In [4]:
meshcat.DeleteAddedControls()


def sigma(x):
    return 1.0 / (1 + np.exp(-x))


def janet(x, w, u, wf, uf, alpha):
    f = sigma(wf * x + uf)
    return -x + f * (1 - alpha) * x + (1 - f) * np.tanh(w * x + u)


Janet = np.vectorize(janet)
xmax = 2.0
ymax = 1.0
x = np.arange(-xmax, xmax, 0.01)

meshcat.Delete()
meshcat.Set2dRenderMode(xmax=xmax, xmin=-xmax, ymin=-ymax, ymax=ymax)
meshcat.SetProperty("/Grid", "visible", True)
meshcat.SetProperty("/Axes", "visible", True)


def update(w=1, u=0, wf=0, uf=0, alpha=0.1):
    # TODO(russt): Visualize fixed points here, too.
    meshcat.SetLine(
        "janet",
        np.vstack([x, 0 * x, Janet(x, w=w, u=u, wf=wf, uf=uf, alpha=alpha)]),
        line_width=4.0,
    )


interact(
    meshcat,
    update,
    w=(0, 3, 0.1),
    u=(-1.5, 1.5, 0.1),
    wf=(-2, 2, 0.1),
    uf=(-5, 5, 0.1),
    alpha=(0, 1, 0.02),
)

Press the 'Stop Interacting' button in Meshcat to continue.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=314062d5-b839-4089-b02f-6c21e42e9581' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>